In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Load dataset
df = pd.read_csv("used_cars.csv")

# Inspect dataset
print(df.info())
print(df.head())

# Handling missing values
df.dropna(inplace=True)  # You can use imputation instead if needed

# Drop unwanted columns
df.drop(columns=['model', 'engine', 'int_col'], inplace=True)

# Ensure 'milage' column is properly formatted
df['milage'] = df['milage'].astype(str).str.replace(r'[,\smi.]', '', regex=True)

# Convert 'milage' to float, handling any conversion errors
df['milage'] = pd.to_numeric(df['milage'], errors='coerce')

# Ensure 'milage' remains numeric before defining X and y
df['milage'] = df['milage'].astype(float)

# Clean 'price' column (remove '$' and commas, convert to float)
df['price'] = df['price'].astype(str).str.replace(r'[$,]', '', regex=True)
df['price'] = pd.to_numeric(df['price'], errors='coerce')

# Verify 'price' conversion
print("\nPrice column after conversion:")
print(df['price'].dtype)
print(df['price'].head())

# Fill missing values in 'clean_title'
df['clean_title'].fillna("No", inplace=True)

# Identify categorical and numerical columns
categorical_cols = ['brand', 'fuel_type', 'accident', 'transmission', 'ext_col', 'clean_title']
numerical_cols = ['model_year', 'milage']

# Ensure categorical columns are strings
df[categorical_cols] = df[categorical_cols].astype(str)

# Fill missing values in categorical columns
df[categorical_cols] = df[categorical_cols].fillna("Unknown")

# Define features and target
X = df.drop(columns=['price'])  # Features
y = df['price']  # Target

# Debugging: Check data types before splitting
print("\nData types before splitting:")
print(X.dtypes)

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Debugging: Check data types and missing values after splitting
print("\nData types before transformation:")
print(X_train.dtypes)
print("\nMissing values before transformation:")
print(X_train.isnull().sum())

# Preprocessing and Model Pipeline
preprocessor = ColumnTransformer([
    ('num', StandardScaler(), numerical_cols),
    ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
])

model_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', LinearRegression())
])

# Train model
model_pipeline.fit(X_train, y_train)

# Predictions
y_pred = model_pipeline.predict(X_test)

# Evaluate model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4009 entries, 0 to 4008
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   brand         4009 non-null   object
 1   model         4009 non-null   object
 2   model_year    4009 non-null   int64 
 3   milage        4009 non-null   object
 4   fuel_type     3839 non-null   object
 5   engine        4009 non-null   object
 6   transmission  4009 non-null   object
 7   ext_col       4009 non-null   object
 8   int_col       4009 non-null   object
 9   accident      3896 non-null   object
 10  clean_title   3413 non-null   object
 11  price         4009 non-null   object
dtypes: int64(1), object(11)
memory usage: 376.0+ KB
None
      brand                            model  model_year      milage  \
0      Ford  Utility Police Interceptor Base        2013  51,000 mi.   
1   Hyundai                     Palisade SEL        2021  34,742 mi.   
2     Lexus                    

/var/folders/vk/39rrm4wn04vdbql5632n168h0000gn/T/ipykernel_37165/2975039311.py:41: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['clean_title'].fillna("No", inplace=True)


In [3]:
print(f"Mean Squared Error: {mse}")
print(f"R-squared Score: {r2}")

print("Model training and evaluation completed.")

Mean Squared Error: 13982950818.646057
R-squared Score: 0.04512762993094577
Model training and evaluation completed.


In [10]:
import pickle
pickle.dump(model_pipeline, open('used_cars_price_model.pkl','wb'))